In [ ]:
import torch
import pandas as pd
import numpy as np
import csv
from copy import copy
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from torch.utils.data import DataLoader, random_split
#from pytorch_lightning.loggers import TensorBoardLogger
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import sys
import os
torch.set_float32_matmul_precision('high')


In [ ]:
outcsv = pd.read_csv('****PUT THE INPUT TRAJECTORY(GIF) DATASET OF YOUR TRAINED NETWORK HERE.csv****', header=None)
Ou = torch.tensor(outcsv.values, dtype=torch.float32)

inpcsv = pd.read_csv('****PUT THE OUTPUT PARAMETER DATASET OF YOUR TRAINED NETWORK HERE.csv****', header=None)
inputs = torch.tensor(inpcsv.values, dtype=torch.float32)#[:, :npara ]

# npara=6

In [ ]:
ntrain =10000
max_lr =1e-3
C1steep=10
bsize_myinput=500
seed=241216
max_epochs=2000
ep_maxlr=1000

outputstGIF = Ou.reshape(Ou.shape[0], 16, 4).transpose(1, 2) 
min_vals = torch.min(inputs[:ntrain], dim=0).values 
max_vals = torch.max(inputs[:ntrain], dim=0).values 
# min_vals = torch.min(inputs_original, dim=0).values 
# max_vals = torch.max(inputs_original, dim=0).values 

inputs_rescaled = (inputs - min_vals) / (max_vals - min_vals)

row_t = outputstGIF[:, 0, :]  # 
row_G = outputstGIF[:, 1, :]  # 
row_I = outputstGIF[:, 2, :]  # 
row_F = outputstGIF[:, 3, :]  # 
row_1onG= torch.reciprocal(row_G)
row_1onI= torch.reciprocal(row_I)
row_1onF= torch.reciprocal(row_F)

n_conv1outchannels=512
n_conv2outchannels=512
n_conv3outchannels=1024
n_dense1=1024
n_dense2=1024

outputs = torch.cat((row_t.unsqueeze(1),row_G.unsqueeze(1),
                     row_t.unsqueeze(1),row_I.unsqueeze(1),
                     row_t.unsqueeze(1),row_F.unsqueeze(1),
                     row_G.unsqueeze(1),row_I.unsqueeze(1),
                     row_I.unsqueeze(1),row_F.unsqueeze(1),
                     row_G.unsqueeze(1),row_F.unsqueeze(1),
                     row_t.unsqueeze(1),row_1onG.unsqueeze(1),
                     row_t.unsqueeze(1),row_1onI.unsqueeze(1),
                     row_t.unsqueeze(1),row_1onF.unsqueeze(1)),dim=1)


print(outputs.shape)
print(min_vals.shape)
print(max_vals.shape)

In [ ]:

def prepare_data_loaders(inputs_rescaled, npara, model_checkpoint, ntrain=ntrain, batch_size=bsize_myinput,  val_split=0.2, num_workers=8):
    
    
    # Load the model from the checkpoint
    if "catonlytGIF" in model_checkpoint or "nocatnoFE" in model_checkpoint or "noFE" in model_checkpoint:
        outputs = torch.cat((row_t.unsqueeze(1),row_G.unsqueeze(1),
                     row_t.unsqueeze(1),row_I.unsqueeze(1),
                     row_t.unsqueeze(1),row_F.unsqueeze(1),
                     row_G.unsqueeze(1),row_I.unsqueeze(1),
                     row_I.unsqueeze(1),row_F.unsqueeze(1),
                     row_G.unsqueeze(1),row_F.unsqueeze(1)),dim=1)
    elif "twithreciGIF" in model_checkpoint:
        outputs = torch.cat((row_t.unsqueeze(1),row_G.unsqueeze(1),
                     row_t.unsqueeze(1),row_I.unsqueeze(1),
                     row_t.unsqueeze(1),row_F.unsqueeze(1),
                     row_G.unsqueeze(1),row_I.unsqueeze(1),
                     row_I.unsqueeze(1),row_F.unsqueeze(1),
                     row_G.unsqueeze(1),row_F.unsqueeze(1),
                     row_t.unsqueeze(1),row_1onG.unsqueeze(1),
                     row_t.unsqueeze(1),row_1onI.unsqueeze(1),
                     row_t.unsqueeze(1),row_1onF.unsqueeze(1)),dim=1)
        
    elif "fullreci" in model_checkpoint:
        outputs = torch.cat((row_t.unsqueeze(1),row_G.unsqueeze(1),
                     row_t.unsqueeze(1),row_I.unsqueeze(1),
                     row_t.unsqueeze(1),row_F.unsqueeze(1),
                     row_G.unsqueeze(1),row_I.unsqueeze(1),
                     row_I.unsqueeze(1),row_F.unsqueeze(1),
                     row_G.unsqueeze(1),row_F.unsqueeze(1),
                     row_t.unsqueeze(1),row_1onG.unsqueeze(1),
                     row_t.unsqueeze(1),row_1onI.unsqueeze(1),
                     row_t.unsqueeze(1),row_1onF.unsqueeze(1),
                     row_1onG.unsqueeze(1),row_1onI.unsqueeze(1),
                     row_1onI.unsqueeze(1),row_1onF.unsqueeze(1),
                     row_1onG.unsqueeze(1),row_1onF.unsqueeze(1)),dim=1)
        
        
    else:
        raise ValueError("Unknown featureEngineering type in checkpoint filename.")

    X = outputs[:ntrain].unsqueeze(1)
#     X_test = outputs[ntrain:].unsqueeze(1)
    X_test = outputs[ntrain:ntrain+ntest].unsqueeze(1)
    
    
    Y = inputs_rescaled[:ntrain, :npara]
#     Y_test = inputs_rescaled[ntrain:, :npara]
    Y_test = inputs_rescaled[ntrain:ntrain+ntest, :npara]
    print("Xshape",X.shape)
    print("Yshape:",Y.shape)
    print("Xtestshape",X_test.shape)
    print("Ytestshape:",Y_test.shape)
    print(X_test[1234])
    print(Y_test[1234])
    
    dataset = TensorDataset(X, Y)
    testset = TensorDataset(X_test, Y_test)
    val_size = int(len(dataset) * val_split)
    train_size = len(dataset) - val_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=torch.Generator().manual_seed(seed))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, generator=torch.Generator().manual_seed(seed))
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return train_loader, val_loader, test_loader



In [ ]:
class Model_post22_cat_tanh01(pl.LightningModule):
    def __init__(self, batch_size,max_lr,conv3size1,npara):#learning_rate):
        super().__init__()
        self.batch_size = batch_size
        self.max_lr=max_lr
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=n_conv1outchannels, kernel_size=(2,2),stride=(2,1))# 
        self.conv2 = nn.Conv2d(in_channels=n_conv1outchannels, out_channels=n_conv2outchannels, kernel_size=(3,2),stride=(3,1))#
        self.replication_pad = nn.ReplicationPad2d((0, 1, 0, 0))
        self.pool1 = nn.MaxPool2d(kernel_size=(1,2))
        self.conv3 = nn.Conv2d(in_channels=n_conv2outchannels, out_channels=n_conv3outchannels, kernel_size=(conv3size1,2))
        self.flatten = nn.Flatten()
        self.dense1 = nn.Linear(3*n_conv3outchannels, n_dense1)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(n_dense1,n_dense2)
        self.dense3 = nn.Linear(n_dense2, npara)
        
    def forward(self, x):       
        x_conv1 = self.conv1(x) 
        x_pad = self.replication_pad(x_conv1) 
        x_rep = x.repeat(1, n_conv2outchannels, 1, 1)
        x_cat= torch.cat([x_rep[:, :, i:i+2, :] if i % 2 == 0 else x_pad[:, :, i//2:i//2+1, :]
               for i in range(x_rep.size(2) + x_pad.size(2))], dim=2) #
        x = self.conv2(x_cat) #12-27
        x = self.pool1(x)
        x = self.conv3(x)
        x = self.pool1(x) 
        x = self.flatten(x)
        x = self.dense1(x)
        x = torch.tanh(x)
        x = self.dense2(x)
        x = torch.tanh(x)
        x = self.dense3(x)
        x = (torch.tanh(x)+1)/2
        return x
    
    
class Model_post22_cat_relu(pl.LightningModule):
    def __init__(self, batch_size,max_lr,conv3size1,npara):#learning_rate):
        super().__init__()
        self.batch_size = batch_size
        self.max_lr=max_lr
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=n_conv1outchannels, kernel_size=(2,2),stride=(2,1))# 
        self.conv2 = nn.Conv2d(in_channels=n_conv1outchannels, out_channels=n_conv2outchannels, kernel_size=(3,2),stride=(3,1))#
        self.replication_pad = nn.ReplicationPad2d((0, 1, 0, 0))
        self.pool1 = nn.MaxPool2d(kernel_size=(1,2))
        self.conv3 = nn.Conv2d(in_channels=n_conv2outchannels, out_channels=n_conv3outchannels, kernel_size=(conv3size1,2))
        self.flatten = nn.Flatten()
        self.dense1 = nn.Linear(3*n_conv3outchannels, n_dense1)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(n_dense1,n_dense2)
        self.dense3 = nn.Linear(n_dense2, npara)
        
    def forward(self, x):       
        x_conv1 = self.conv1(x) 
        x_pad = self.replication_pad(x_conv1) 
        x_rep = x.repeat(1, n_conv2outchannels, 1, 1)
        x_cat= torch.cat([x_rep[:, :, i:i+2, :] if i % 2 == 0 else x_pad[:, :, i//2:i//2+1, :]
               for i in range(x_rep.size(2) + x_pad.size(2))], dim=2) #
        x = self.conv2(x_cat) 
        x = self.pool1(x) 
        x = self.conv3(x) 
        x = self.pool1(x)     
        x = self.flatten(x)
        x = self.dense1(x)
        x = torch.tanh(x)
        x = self.dense2(x)
        x = torch.tanh(x)
        x = self.dense3(x)
        x = self.relu(x)
        return x
    
    
class Model_post22_nocat_tanh01(pl.LightningModule):
    def __init__(self, batch_size,max_lr,conv3size1,npara):#learning_rate):
        super().__init__()
        self.batch_size = batch_size
        self.max_lr=max_lr
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=n_conv1outchannels, kernel_size=(2,2),stride=(2,1))# 
        self.conv2 = nn.Conv2d(in_channels=n_conv1outchannels, out_channels=n_conv2outchannels, kernel_size=(3,2),stride=(3,1))#
        self.replication_pad = nn.ReplicationPad2d((0, 1, 0, 0))
        self.pool1 = nn.MaxPool2d(kernel_size=(1,2))
        self.conv3 = nn.Conv2d(in_channels=n_conv2outchannels, out_channels=n_conv3outchannels, kernel_size=(conv3size1,2))
        self.flatten = nn.Flatten()
        self.dense1 = nn.Linear(3*n_conv3outchannels, n_dense1)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(n_dense1,n_dense2)
        self.dense3 = nn.Linear(n_dense2, npara)
        
    def forward(self, x):       
        x_conv1 = self.conv1(x) 
        x = self.pool1(x_conv1) 
        x = self.conv3(x) 
        x = self.pool1(x)   
        x = self.flatten(x)
        x = self.dense1(x)
        x = torch.tanh(x)
        x = self.dense2(x)
        x = torch.tanh(x)
        x = self.dense3(x)
        x = (torch.tanh(x)+1)/2
        return x
    
    
class Model_post22_nocat_relu(pl.LightningModule):
    def __init__(self, batch_size,max_lr,conv3size1,npara):#learning_rate):
        super().__init__()
        self.batch_size = batch_size
        self.max_lr=max_lr
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=n_conv1outchannels, kernel_size=(2,2),stride=(2,1))# 
        self.conv2 = nn.Conv2d(in_channels=n_conv1outchannels, out_channels=n_conv2outchannels, kernel_size=(3,2),stride=(3,1))#
        self.replication_pad = nn.ReplicationPad2d((0, 1, 0, 0))
        self.pool1 = nn.MaxPool2d(kernel_size=(1,2))
        self.conv3 = nn.Conv2d(in_channels=n_conv2outchannels, out_channels=n_conv3outchannels, kernel_size=(conv3size1,2))
        self.flatten = nn.Flatten()
        self.dense1 = nn.Linear(3*n_conv3outchannels, n_dense1)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(n_dense1,n_dense2)
        self.dense3 = nn.Linear(n_dense2, npara)
        
    def forward(self, x):       
        x_conv1 = self.conv1(x) 
        x = self.pool1(x_conv1) 
        x = self.conv3(x)
        x = self.pool1(x)   
        x = self.flatten(x)
        x = self.dense1(x)
        x = torch.tanh(x)
        x = self.dense2(x)
        x = torch.tanh(x)
        x = self.dense3(x)
        x = self.relu(x)
        return x

In [ ]:
### USE This block if you want to see the inference of trained network over training set

def infertrain_and_plot(model_checkpoint, min_vals, 
                         max_vals, param_names, batch_size, max_lr):
    
    if "catonlytGIF" in model_checkpoint or "nocatnoFE" in model_checkpoint or "noFE" in model_checkpoint:
        conv3size1_real=6
    elif "twithreciGIF" in model_checkpoint:
        conv3size1_real=9
    elif "fullreci" in model_checkpoint:
        conv3size1_real=12
    else:
        raise ValueError("Unknown featureEngineering type in checkpoint filename.")

#     npara_real=npara
    if "noinferGIFb" in model_checkpoint or "npara6" in model_checkpoint:
        npara_real=6
    elif "inferGIFb" in model_checkpoint or "npara8" in model_checkpoint:  
        npara_real=8
    elif "inferGFbGF22" in model_checkpoint or "npara10" in model_checkpoint:
        npara_real=10
    else:
        raise ValueError("Unknown Number of Parameters to be inferred in checkpoint filename.")
    
    train_loader, val_loader, test_loader=prepare_data_loaders(inputs_rescaled=inputs_rescaled,npara=npara_real, model_checkpoint=model_checkpoint)
                  
    # Load the model from the checkpoint
    if "relu" in model_checkpoint:
        
        if "nocatnoFE" in model_checkpoint or "noFE" in model_checkpoint:
            model_1 = Model_post22_nocat_relu.load_from_checkpoint(
                checkpoint_path=model_checkpoint,
                batch_size=batch_size,max_lr=max_lr,conv3size1=conv3size1_real,npara=npara_real,
                map_location=torch.device('cpu')
            )
        else:
            model_1 = Model_post22_cat_relu.load_from_checkpoint(
                checkpoint_path=model_checkpoint,
                batch_size=batch_size,max_lr=max_lr,conv3size1=conv3size1_real,npara=npara_real,
                map_location=torch.device('cpu')
            )    
                
    elif "tanh01" in model_checkpoint:
        if "nocatnoFE" in model_checkpoint or "noFE" in model_checkpoint:           
            model_1 = Model_post22_nocat_tanh01.load_from_checkpoint(
                checkpoint_path=model_checkpoint,
                batch_size=batch_size,max_lr=max_lr,conv3size1=conv3size1_real,npara=npara_real,
                map_location=torch.device('cpu')
            )                  
        else:
            model_1 = Model_post22_cat_tanh01.load_from_checkpoint(
                checkpoint_path=model_checkpoint,
                batch_size=batch_size,max_lr=max_lr,conv3size1=conv3size1_real,npara=npara_real,
                map_location=torch.device('cpu')
            )
                
    else:
        raise ValueError("Unknown model type in checkpoint filename.")

    model_1.eval()
    
    # Prepare to store the model outputs and correct answers
    all_outputs = []
    all_correct_answers = []
    num_samples = 0  # To keep track of the number of processed samples

    with torch.no_grad():
        for batch in train_loader:
            inputs, correct_answers = batch  # Assuming train_loader yields (input, target) pairs
            print(inputs.shape)
            print(correct_answers.shape)
            outputs = model_1(inputs)
            
            all_outputs.append(outputs)
            all_correct_answers.append(correct_answers)
            num_samples += inputs.size(0)

            if num_samples >= 10000:
                break  # Stop once we've processed 10,00 samples

    all_outputs = torch.cat(all_outputs, dim=0)[:10000]
    all_correct_answers = torch.cat(all_correct_answers, dim=0)[:10000]
    op_rescaledback = all_outputs  * (max_vals[:npara_real] - min_vals[:npara_real]) + min_vals[:npara_real]
    all_correct_answers = all_correct_answers * (max_vals[:npara_real] - min_vals[:npara_real]) + min_vals[:npara_real]
    op_rescaledback_np = op_rescaledback.numpy()
    correct_answers_np = all_correct_answers.numpy()

    # Plot the results
    fig, axes = plt.subplots(3, 4, figsize=(15, 15))
    axes = axes.flatten()

    # Set font size for titles, labels, and ticks
    title_fontsize = 20
    label_fontsize = 14
    tick_fontsize = 12

    for i in range(npara_real):
        axes[i].scatter(correct_answers_np[:, i], op_rescaledback_np[:, i], alpha=0.5)
        axes[i].set_xlabel(f"True {param_names[i]}", fontsize=label_fontsize)
        axes[i].set_ylabel(f"Inferred {param_names[i]}", fontsize=label_fontsize)
        axes[i].set_title(f"{param_names[i]}", fontsize=title_fontsize)
        axes[i].plot([min(correct_answers_np[:, i]), max(correct_answers_np[:, i])], 
                     [min(correct_answers_np[:, i]), max(correct_answers_np[:, i])], 
                     'r--')  # Add a diagonal line for reference

#         for j in range(correct_answers_np.shape[0]):
#             axes[i].annotate(f'{j+1}', (correct_answers_np[j, i], op_rescaledback_np[j, i]),
#                              textcoords="offset points", xytext=(5,-5), ha='center', fontsize=8)
            
        # Adjust tick label font size
        axes[i].tick_params(axis='both', which='major', labelsize=tick_fontsize)

    plt.subplots_adjust(hspace=0.4, wspace=0.4)
    plt.show()
    return correct_answers_np,op_rescaledback_np

param_names = ['$S_I$','$C_X$','$S_G$','$X_2$','$C_F$','$L_2$','$G_b$','$F_b$','G22','F22']

### IMPORT your model_checkpoint here!
### It is in the form of, for example:
# model_checkpoint='model_GFX3DNN_catonlytGIF_mvlognmdirect_npara10_tanh01_241216_25020328_bestepoch=1213-val_loss=0.00003.ckpt'

paratrain,parainfertrain=infertrain_and_plot(model_checkpoint, min_vals, 
                         max_vals, param_names, batch_size=bsize_myinput, max_lr=max_lr)

### Example for saving:
#np.savetxt("paratrain_Mvlognormaldirect_jobid.csv", paratrain, delimiter=",") 
#np.savetxt("parainfertrain_Mvlognormaldirect_jobid.csv", parainfertrain, delimiter=",") 


In [ ]:
### USE This block if you want to see the inference of trained network over the TESTING set

def infertest_and_plot(model_checkpoint, min_vals, 
                         max_vals, param_names, batch_size, max_lr):
    
    if "catonlytGIF" in model_checkpoint or "nocatnoFE" in model_checkpoint or "noFE" in model_checkpoint:
        conv3size1_real=6
    elif "twithreciGIF" in model_checkpoint:
        conv3size1_real=9
    elif "fullreci" in model_checkpoint:
        conv3size1_real=12
    else:
        raise ValueError("Unknown featureEngineering type in checkpoint filename.")

#     npara_real=npara
    if "noinferGIFb" in model_checkpoint or "npara6" in model_checkpoint:
        npara_real=6
    elif "inferGIFb" in model_checkpoint or "npara8" in model_checkpoint:  
        npara_real=8
    elif "inferGFbGF22" in model_checkpoint or "npara10" in model_checkpoint:
        npara_real=10
    else:
        raise ValueError("Unknown Number of Parameters to be inferred in checkpoint filename.")
    
    train_loader, val_loader, test_loader=prepare_data_loaders(inputs_rescaled=inputs_rescaled,npara=npara_real, model_checkpoint=model_checkpoint)
                  
    # Load the model from the checkpoint
    if "relu" in model_checkpoint:
        
        if "nocatnoFE" in model_checkpoint or "noFE" in model_checkpoint:
            model_1 = Model_post22_nocat_relu.load_from_checkpoint(
                checkpoint_path=model_checkpoint,
                batch_size=batch_size,max_lr=max_lr,conv3size1=conv3size1_real,npara=npara_real,
                map_location=torch.device('cpu')
            )
        else:
            model_1 = Model_post22_cat_relu.load_from_checkpoint(
                checkpoint_path=model_checkpoint,
                batch_size=batch_size,max_lr=max_lr,conv3size1=conv3size1_real,npara=npara_real,
                map_location=torch.device('cpu')
            )    
                
    elif "tanh01" in model_checkpoint:
        if "nocatnoFE" in model_checkpoint or "noFE" in model_checkpoint:           
            model_1 = Model_post22_nocat_tanh01.load_from_checkpoint(
                checkpoint_path=model_checkpoint,
                batch_size=batch_size,max_lr=max_lr,conv3size1=conv3size1_real,npara=npara_real,
                map_location=torch.device('cpu')
            )                  
        else:
            model_1 = Model_post22_cat_tanh01.load_from_checkpoint(
                checkpoint_path=model_checkpoint,
                batch_size=batch_size,max_lr=max_lr,conv3size1=conv3size1_real,npara=npara_real,
                map_location=torch.device('cpu')
            )
                
    else:
        raise ValueError("Unknown model type in checkpoint filename.")

    model_1.eval()
    
    # Prepare to store the model outputs and correct answers
    all_outputs = []
    all_correct_answers = []
    num_samples = 0  # To keep track of the number of processed samples

    with torch.no_grad():
        for batch in test_loader:
            inputs, correct_answers = batch  # Assuming train_loader yields (input, target) pairs
            print(inputs.shape)
            print(correct_answers.shape)
            outputs = model_1(inputs)
            
            all_outputs.append(outputs)
            all_correct_answers.append(correct_answers)
            num_samples += inputs.size(0)

            if num_samples >= 10000:
                break  # Stop once we've processed 10,00 samples

    all_outputs = torch.cat(all_outputs, dim=0)[:10000]
    all_correct_answers = torch.cat(all_correct_answers, dim=0)[:10000]
    op_rescaledback = all_outputs  * (max_vals[:npara_real] - min_vals[:npara_real]) + min_vals[:npara_real]
    all_correct_answers = all_correct_answers * (max_vals[:npara_real] - min_vals[:npara_real]) + min_vals[:npara_real]
    op_rescaledback_np = op_rescaledback.numpy()
    correct_answers_np = all_correct_answers.numpy()

    # Plot the results
    fig, axes = plt.subplots(3, 4, figsize=(15, 15))
    axes = axes.flatten()

    # Set font size for titles, labels, and ticks
    title_fontsize = 20
    label_fontsize = 14
    tick_fontsize = 12

    for i in range(npara_real):
        axes[i].scatter(correct_answers_np[:, i], op_rescaledback_np[:, i], alpha=0.5)
        axes[i].set_xlabel(f"True {param_names[i]}", fontsize=label_fontsize)
        axes[i].set_ylabel(f"Inferred {param_names[i]}", fontsize=label_fontsize)
        axes[i].set_title(f"{param_names[i]}", fontsize=title_fontsize)
        axes[i].plot([min(correct_answers_np[:, i]), max(correct_answers_np[:, i])], 
                     [min(correct_answers_np[:, i]), max(correct_answers_np[:, i])], 
                     'r--')  # Add a diagonal line for reference

#         for j in range(correct_answers_np.shape[0]):
#             axes[i].annotate(f'{j+1}', (correct_answers_np[j, i], op_rescaledback_np[j, i]),
#                              textcoords="offset points", xytext=(5,-5), ha='center', fontsize=8)
            
        # Adjust tick label font size
        axes[i].tick_params(axis='both', which='major', labelsize=tick_fontsize)

    plt.subplots_adjust(hspace=0.4, wspace=0.4)
    plt.show()
    return correct_answers_np,op_rescaledback_np




# param_names = ['$S_I$','$C_X$','$S_G$','$C_Y$','$X_2$','$C_F$','$L_2$','$yt_{22}$','$G_b$','$F_b$','Initial condition of $G$','Initial condition of $F$']
param_names = ['$S_I$','$C_X$','$S_G$','$X_2$','$C_F$','$L_2$','$G_b$','$F_b$','G22','F22']

 
### USE This block if you want to see the inference of trained network over the TESTING set

def infertest_and_plot(model_checkpoint, ntrain, param_names, batch_size, max_lr):
    # Determine feature engineering size
    if "catonlytGIF" in model_checkpoint or "noFE" in model_checkpoint:
        conv3size1_real = 6
    elif "twithreciGIF" in model_checkpoint:
        conv3size1_real = 9
    elif "fullreci" in model_checkpoint:
        conv3size1_real = 12
    else:
        raise ValueError("Unknown featureEngineering type in checkpoint filename.")

    # Prepare data loaders
    train_loader, val_loader, test_loader = prepare_data_loaders(
        npara=npara_real,
        model_checkpoint=model_checkpoint,
        ntrain=ntrain
    )

    # Load model
    if "relu" in model_checkpoint:
        ModelClass = Model2D_post22_nocat_relu if "noFE" in model_checkpoint else Model2D_post22_cat_relu
    elif "tanh01" in model_checkpoint:
        ModelClass = Model2D_post22_nocat_tanh01 if "noFE" in model_checkpoint else Model2D_post22_cat_tanh01
    else:
        raise ValueError("Unknown model type in checkpoint filename.")

    model_1 = ModelClass.load_from_checkpoint(
        checkpoint_path=model_checkpoint,
        batch_size=batch_size,
        max_lr=max_lr,
        conv3size1=conv3size1_real,
        npara=npara_real,
        map_location=torch.device('cpu')
    )
    model_1.eval()

    # Compute min/max for rescaling
    min_vals = torch.min(inputs[:ntrain], dim=0).values
    max_vals = torch.max(inputs[:ntrain], dim=0).values

    # Inference loop
    all_outputs = []
    all_correct = []
    samples = 0
    with torch.no_grad():
        for ip, correct in test_loader:
            out = model_1(ip)
            all_outputs.append(out)
            all_correct.append(correct)
            samples += ip.size(0)
            if samples >= 10000:
                break

    # Concatenate and trim
    all_outputs = torch.cat(all_outputs, dim=0)[:10000]
    all_correct = torch.cat(all_correct, dim=0)[:10000]

    # Rescale back to original
    scale = max_vals[:npara_real] - min_vals[:npara_real]
    op_rescaled = all_outputs * scale + min_vals[:npara_real]
    correct_rescaled = all_correct * scale + min_vals[:npara_real]

    return correct_rescaled.numpy(), op_rescaled.numpy()


### IMPORT your model_checkpoint here!
### It is in the form of, for example:
# model_checkpoint='model_GFX3DNN_catonlytGIF_mvlognmdirect_npara10_tanh01_241216_25020328_bestepoch=1213-val_loss=0.00003.ckpt'

paratest,parainfertest=infertest_and_plot(model_checkpoint, min_vals, 
                         max_vals, param_names, batch_size=bsize_myinput, max_lr=max_lr)

### Example for saving:
#np.savetxt("paratest_Mvlognormaldirect_jobid.csv", paratest, delimiter=",") 
#np.savetxt("parainfertest_Mvlognormaldirect_jobid.csv", parainfertest, delimiter=",") 

In [ ]:

param_names = ['$S_I$','$C_X$','$S_G$','$X_2$','$C_F$','$L_2$','$G_b$','$F_b$','G22','F22']


FFA = pd.read_csv('GFX3D25_optmF.csv', header=None).values
Glu = pd.read_csv('GFX3D25_optmG.csv', header=None).values

# denoiseFFA=pd.read_csv('FCdenoised_FFA_for250115_3930694data.csv', header=None).values
Ins   = pd.read_csv('25FSIGT_Ins.csv', header=None).values

correct_answers3Doptm = torch.tensor(pd.read_csv('GFX3Doptm_correctanswers.csv', header=None).values, dtype=torch.float32)

tvec = [22,  24, 25, 27, 30, 40, 50, 60, 70, 80, 90, 100, 120, 140,160, 180]
optmdata25 = np.zeros((25, 4, 16))
optmdata25[:, 0, :] = tvec
optmdata25[:, 1, :] = Glu
optmdata25[:, 2, :] = Ins
optmdata25[:, 3, :] = FFA
optmdata25_tensor = torch.tensor(optmdata25, dtype=torch.float32)

row_t_optm = optmdata25_tensor[:, 0, :]
row_G_optm = optmdata25_tensor[:, 1, :]
row_I_optm = optmdata25_tensor[:, 2, :]
row_F_optm = optmdata25_tensor[:, 3, :]
row_1onG_optm = torch.reciprocal(row_G_optm)
row_1onI_optm = torch.reciprocal(row_I_optm)
row_1onF_optm = torch.reciprocal(row_F_optm)

    
def inferoptm_and_plot(correct_answers, model_checkpoint, min_vals,
                         max_vals, param_names, batch_size=bsize_myinput, max_lr=max_lr):


    if "catonlytGIF" in model_checkpoint or "noFE" in model_checkpoint or "nocatnoFE" in model_checkpoint:
        optmdata25_FEinputs = torch.cat((row_t_optm.unsqueeze(1), row_G_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_I_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_F_optm.unsqueeze(1),
                                 row_G_optm.unsqueeze(1), row_I_optm.unsqueeze(1),
                                 row_I_optm.unsqueeze(1), row_F_optm.unsqueeze(1),
                                 row_G_optm.unsqueeze(1), row_F_optm.unsqueeze(1)), dim=1).unsqueeze(1)
#                                      row_t.unsqueeze(1), row_1onG.unsqueeze(1),
#                                      row_t.unsqueeze(1), row_1onI.unsqueeze(1),
#                                      row_t.unsqueeze(1), row_1onF.unsqueeze(1)), dim=1).unsqueeze(1)


        conv3size1_real=6
    elif "twithreciGIF" in model_checkpoint:
        conv3size1_real=9
        optmdata25_FEinputs = torch.cat((row_t_optm.unsqueeze(1), row_G_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_I_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_F_optm.unsqueeze(1),
                                 row_G_optm.unsqueeze(1), row_I_optm.unsqueeze(1),
                                 row_I_optm.unsqueeze(1), row_F_optm.unsqueeze(1),
                                 row_G_optm.unsqueeze(1), row_F_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_1onG_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_1onI_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_1onF_optm.unsqueeze(1)), dim=1).unsqueeze(1)

    elif "fullreci" in model_checkpoint:
        conv3size1_real=12
        optmdata25_FEinputs = torch.cat((row_t_optm.unsqueeze(1), row_G_optm.unsqueeze(1),
                         row_t_optm.unsqueeze(1), row_I_optm.unsqueeze(1),
                         row_t_optm.unsqueeze(1), row_F_optm.unsqueeze(1),
                         row_G_optm.unsqueeze(1), row_I_optm.unsqueeze(1),
                         row_I_optm.unsqueeze(1), row_F_optm.unsqueeze(1),
                         row_G_optm.unsqueeze(1), row_F_optm.unsqueeze(1),
                         row_t_optm.unsqueeze(1), row_1onG_optm.unsqueeze(1),
                         row_t_optm.unsqueeze(1), row_1onI_optm.unsqueeze(1),
                         row_t_optm.unsqueeze(1), row_1onF_optm.unsqueeze(1),
                         row_1onG_optm.unsqueeze(1), row_1onI_optm.unsqueeze(1),
                         row_1onI_optm.unsqueeze(1), row_1onF_optm.unsqueeze(1),
                         row_1onG_optm.unsqueeze(1), row_1onF_optm.unsqueeze(1)), dim=1).unsqueeze(1)

        
        
        
        
    else:
        raise ValueError("Unknown featureEngineering type in checkpoint filename.")

        
        
    dataset_denoisedFFA = TensorDataset(optmdata25_FEinputs)
    denoisedFFA_loader = DataLoader(dataset_denoisedFFA, batch_size=batch_size, shuffle=False)

    if "noinferGIFb" in model_checkpoint or "npara6" in model_checkpoint:
        npara_real=6
    elif "inferGIFb" in model_checkpoint or "npara8" in model_checkpoint:  
        npara_real=8
    elif "inferGFbGF22" in model_checkpoint or "npara10" in model_checkpoint:
        npara_real=10
    else:
        raise ValueError("Unknown Number of Parameters to be inferred in checkpoint filename.")
    
    train_loader, val_loader, test_loader=prepare_data_loaders(inputs_rescaled=inputs_rescaled,npara=npara_real, model_checkpoint=model_checkpoint)
                  
    # Load the model from the checkpoint
    if "relu" in model_checkpoint:
        
        if "nocatnoFE" in model_checkpoint or "noFE" in model_checkpoint:
            model_1 = Model_post22_nocat_relu.load_from_checkpoint(
                checkpoint_path=model_checkpoint,
                batch_size=batch_size,max_lr=max_lr,conv3size1=conv3size1_real,npara=npara_real,
                map_location=torch.device('cpu')
            )
        else:
            model_1 = Model_post22_cat_relu.load_from_checkpoint(
                checkpoint_path=model_checkpoint,
                batch_size=batch_size,max_lr=max_lr,conv3size1=conv3size1_real,npara=npara_real,
                map_location=torch.device('cpu')
            )    
                
    elif "tanh01" in model_checkpoint:
        if "nocatnoFE" in model_checkpoint or "noFE" in model_checkpoint:           
            model_1 = Model_post22_nocat_tanh01.load_from_checkpoint(
                checkpoint_path=model_checkpoint,
                batch_size=batch_size,max_lr=max_lr,conv3size1=conv3size1_real,npara=npara_real,
                map_location=torch.device('cpu')
            )                  
        else:
            model_1 = Model_post22_cat_tanh01.load_from_checkpoint(
                checkpoint_path=model_checkpoint,
                batch_size=batch_size,max_lr=max_lr,conv3size1=conv3size1_real,npara=npara_real,
                map_location=torch.device('cpu')
            )
                
    else:
        raise ValueError("Unknown model type in checkpoint filename.")


    model_1.eval()
    with torch.no_grad():
        for batch in denoisedFFA_loader:
            ip = batch[0]
            op = model_1(ip)
            break  # Only one batch is needed 
          
    print(op.shape)
    # Rescale the output back to the original scale
    op_rescaledback = op * (max_vals[:npara_real] - min_vals[:npara_real]) + min_vals[:npara_real]
    print(max_vals.shape)
    # Convert to numpy for easier plotting
    op_rescaledback_np = op_rescaledback.numpy()
    correct_answers_np = correct_answers.numpy()
    print(correct_answers_np.shape)

    fig, axes = plt.subplots(3,4, figsize=(15, 15))
    axes = axes.flatten()
    title_fontsize = 20
    label_fontsize = 14
    tick_fontsize = 12
    
    # Indices of subjects to exclude
#     exclude_indices = [2,10,17,24]  # Python uses zero-based indexing, so subjects 4 and 25 correspond to indices 3 and 24

    # Filter out the excluded subjects
    filtered_correct_answers =correct_answers_np# np.delete(correct_answers_np, exclude_indices, axis=0)#
    filtered_op_rescaledback =op_rescaledback_np#np.delete(op_rescaledback_np, exclude_indices, axis=0)# 
    
    
    for i in range(npara_real):
        axes[i].scatter(filtered_correct_answers[:, i], filtered_op_rescaledback[:, i], alpha=0.5)
        axes[i].set_xlabel(f"True {param_names[i]}", fontsize=label_fontsize)
        axes[i].set_ylabel(f"Inferred {param_names[i]}", fontsize=label_fontsize)
        axes[i].set_title(f"{param_names[i]}", fontsize=title_fontsize)
        axes[i].plot([min(filtered_correct_answers[:, i]), max(filtered_correct_answers[:, i])], 
                     [min(filtered_correct_answers[:, i]), max(filtered_correct_answers[:, i])], 
                     'r--')  # Add a diagonal line for reference
        # Annotate remaining points
        for j in range(filtered_correct_answers.shape[0]):
            axes[i].annotate(f'{j+1}', (filtered_correct_answers[j, i], filtered_op_rescaledback[j, i]),
                             textcoords="offset points", xytext=(5, -5), ha='center', fontsize=8)

        # Adjust tick label font size
        axes[i].tick_params(axis='both', which='major', labelsize=tick_fontsize)

    plt.subplots_adjust(hspace=0.4, wspace=0.4)
    plt.show()
    return filtered_op_rescaledback



### USE This block if you want to see the inference of trained network from the MODEL-FITTING CURVE from optimization



# correct_answers = torch.tensor(pd.read_csv('New75FSIGT_optmpara.csv', header=None).values, dtype=torch.float32)
FFA = pd.read_csv('FX2D25_optmF.csv', header=None).values
Glu = pd.read_csv('25FSIGT_glu_alltime.csv', header=None).values
# denoiseFFA=pd.read_csv('FX2D25_FCdenoised_2877269datafirst500k_250203.csv', header=None).values
Ins   = pd.read_csv('25FSIGT_ins_alltime.csv', header=None).values

correct_answers2Doptm = torch.tensor(pd.read_csv('FX2D25_paraoptm.csv', header=None).values, dtype=torch.float32)


tvec =  [0,   2,   3,   4,   5,   6,   8,  10,  12,  14,  16,  19, 22,   24, 25, 27, 30, 40, 50, 60, 70, 80, 90, 100, 120, 140,160, 180]
# tvec = [22,  24, 25, 27, 30, 40, 50, 60, 70, 80, 90, 100, 120, 140,160, 180]
realdata25 = np.zeros((25, 4, 28))
realdata25[:, 0, :] = tvec
realdata25[:, 1, :] = Glu
realdata25[:, 2, :] = Ins
realdata25[:, 3, :] = FFA
realdata25_tensor = torch.tensor(realdata25, dtype=torch.float32)

row_t_optm = realdata25_tensor[:, 0, :]
row_G_optm = realdata25_tensor[:, 1, :]
row_I_optm = realdata25_tensor[:, 2, :]
row_F_optm = realdata25_tensor[:, 3, :]
row_1onG_optm = torch.reciprocal(row_G_optm)
row_1onI_optm = torch.reciprocal(row_I_optm)
row_1onF_optm = torch.reciprocal(row_F_optm)

    
def inferoptm_and_plot(correct_answers, model_checkpoint, ntrain, param_names, batch_size=bsize_myinput, max_lr=max_lr):


    if "catonlytGIF" in model_checkpoint:
        realdata25_FEinputs = torch.cat((row_t_optm.unsqueeze(1), row_G_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_I_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_F_optm.unsqueeze(1),
                                 row_G_optm.unsqueeze(1), row_I_optm.unsqueeze(1),
                                 row_I_optm.unsqueeze(1), row_F_optm.unsqueeze(1),
                                 row_G_optm.unsqueeze(1), row_F_optm.unsqueeze(1)), dim=1).unsqueeze(1)
#                                      row_t.unsqueeze(1), row_1onG.unsqueeze(1),
#                                      row_t.unsqueeze(1), row_1onI.unsqueeze(1),
#                                      row_t.unsqueeze(1), row_1onF.unsqueeze(1)), dim=1).unsqueeze(1)
        conv3size1_real=6
    elif "noFE" in model_checkpoint:
        realdata25_FEinputs = torch.cat((row_t_optm.unsqueeze(1), row_G_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_I_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_F_optm.unsqueeze(1),
                                 row_G_optm.unsqueeze(1), row_I_optm.unsqueeze(1),
                                 row_I_optm.unsqueeze(1), row_F_optm.unsqueeze(1),
                                 row_G_optm.unsqueeze(1), row_F_optm.unsqueeze(1)), dim=1).unsqueeze(1)


        conv3size1_real=6
    elif "twithreciGIF" in model_checkpoint:
        conv3size1_real=9
        realdata25_FEinputs = torch.cat((row_t_optm.unsqueeze(1), row_G_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_I_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_F_optm.unsqueeze(1),
                                 row_G_optm.unsqueeze(1), row_I_optm.unsqueeze(1),
                                 row_I_optm.unsqueeze(1), row_F_optm.unsqueeze(1),
                                 row_G_optm.unsqueeze(1), row_F_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_1onG_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_1onI_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_1onF_optm.unsqueeze(1)), dim=1).unsqueeze(1)
        
    elif "fullreci" in model_checkpoint:
        conv3size1_real=12
        realdata25_FEinputs = torch.cat((row_t_optm.unsqueeze(1), row_G_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_I_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_F_optm.unsqueeze(1),
                                 row_G_optm.unsqueeze(1), row_I_optm.unsqueeze(1),
                                 row_I_optm.unsqueeze(1), row_F_optm.unsqueeze(1),
                                 row_G_optm.unsqueeze(1), row_F_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_1onG_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_1onI_optm.unsqueeze(1),
                                 row_t_optm.unsqueeze(1), row_1onF_optm.unsqueeze(1),
                                 row_1onG_optm.unsqueeze(1), row_1onI_optm.unsqueeze(1),
                                 row_1onI_optm.unsqueeze(1), row_1onF_optm.unsqueeze(1),
                                 row_1onG_optm.unsqueeze(1), row_1onF_optm.unsqueeze(1)), dim=1).unsqueeze(1)

    else:
        raise ValueError("Unknown featureEngineering type in checkpoint filename.")

    dataset_denoisedFFA = TensorDataset(realdata25_FEinputs)
    denoisedFFA_loader = DataLoader(dataset_denoisedFFA, batch_size=batch_size, shuffle=False)
    print(npara_real)

    # Load the model from the checkpoint
    if "relu" in model_checkpoint:
        
        if "noFE" in model_checkpoint or "nocatnoFE" in model_checkpoint:
            model_1 = Model2D_post22_nocat_relu.load_from_checkpoint(
                checkpoint_path=model_checkpoint,
                batch_size=batch_size,max_lr=max_lr,conv3size1=conv3size1_real,npara=npara_real,
                map_location=torch.device('cpu')
            )
        else:
            model_1 = Model2D_post22_cat_relu.load_from_checkpoint(
                checkpoint_path=model_checkpoint,
                batch_size=batch_size,max_lr=max_lr,conv3size1=conv3size1_real,npara=npara_real,
                map_location=torch.device('cpu')
            )    
                
    elif "tanh01" in model_checkpoint:
        if "noFE" in model_checkpoint or "nocatnoFE" in model_checkpoint:           
            model_1 = Model2D_post22_nocat_tanh01.load_from_checkpoint(
                checkpoint_path=model_checkpoint,
                batch_size=batch_size,max_lr=max_lr,conv3size1=conv3size1_real,npara=npara_real,
                map_location=torch.device('cpu')
            )                  
        else:
            model_1 = Model2D_post22_cat_tanh01.load_from_checkpoint(
                checkpoint_path=model_checkpoint,
                batch_size=batch_size,max_lr=max_lr,conv3size1=conv3size1_real,npara=npara_real,
                map_location=torch.device('cpu')
            )
                
    else:
        raise ValueError("Unknown model type in checkpoint filename.")


    model_1.eval()
    min_vals = torch.min(inputs[:ntrain], dim=0).values 
    max_vals = torch.max(inputs[:ntrain], dim=0).values 
    
    with torch.no_grad():
        for batch in denoisedFFA_loader:
            ip = batch[0]
            op = model_1(ip)
            break  # Only one batch is needed 
          
    print(op.shape)
    # Rescale the output back to the original scale
    op_rescaledback = op * (max_vals[:npara_real] - min_vals[:npara_real]) + min_vals[:npara_real]
    print(max_vals.shape)
    # Convert to numpy for easier plotting
    op_rescaledback_np = op_rescaledback.numpy()
    correct_answers_np = correct_answers.numpy()
    print(correct_answers_np.shape)

    
    # Indices of subjects to exclude
#     exclude_indices = [2,10,17,24]  # Python uses zero-based indexing, so subjects 4 and 25 correspond to indices 3 and 24

    # Filter out the excluded subjects
    filtered_correct_answers =correct_answers_np# np.delete(correct_answers_np, exclude_indices, axis=0)#
    filtered_op_rescaledback =op_rescaledback_np#np.delete(op_rescaledback_np, exclude_indices, axis=0)# 
    

    return filtered_op_rescaledback

### IMPORT your model_checkpoint here!
### It is in the form of, for example:
# model_checkpoint='model_GFX3DNN_catonlytGIF_mvlognmdirect_npara10_tanh01_241216_25020328_bestepoch=1213-val_loss=0.00003.ckpt'


inferoptm = inferoptm_and_plot(correct_answers, model_checkpoint, min_vals, max_vals, param_names)
### Example for saving:
#np.savetxt("parainferfromoptm_Mvlognormaldirect_jobid.csv", inferoptm, delimiter=",")
